In [2]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose as season
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn import metrics
from sklearn.linear_model import LinearRegression as LinReg

In [25]:
df = pd.read_csv("zillow_data.csv")
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [26]:
df.shape

(14723, 272)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [28]:
df_zip = df.set_index(df["RegionName"])

In [33]:
df_zip = df_zip.drop(columns=["RegionID", "RegionName", "City", "State", "Metro", "CountyName", "SizeRank"])

In [39]:
df_zip.head()

,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
RegionName,,,,,,,,,,,,,,,,,,,,,
60657,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
75070,235700.0,236900.0,236700.0,235400.0,233300.0,230600.0,227300.0,223400.0,219600.0,215800.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
77494,210400.0,212200.0,212200.0,210700.0,208300.0,205500.0,202500.0,199800.0,198300.0,197300.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
60614,498100.0,500900.0,503100.0,504600.0,505500.0,505700.0,505300.0,504200.0,503600.0,503400.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
79936,77300.0,77300.0,77300.0,77300.0,77400.0,77500.0,77600.0,77700.0,77700.0,77800.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [38]:
df_zip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14723 entries, 60657 to 89155
Columns: 265 entries, 1996-04 to 2018-04
dtypes: float64(219), int64(46)
memory usage: 30.5 MB


In [34]:
df_t = df_zip.transpose()

In [35]:
df_t.head(20)

RegionName,60657,75070,77494,60614,79936,77084,10467,60640,77449,94109,...,3765,84781,12429,97028,12720,1338,3293,40404,81225,89155
1996-04,334200.0,235700.0,210400.0,498100.0,77300.0,95000.0,152900.0,216500.0,95400.0,766000.0,...,80800.0,135900.0,78300.0,136200.0,62500.0,94600.0,92700.0,57100.0,191100.0,176400.0
1996-05,335400.0,236900.0,212200.0,500900.0,77300.0,95200.0,152700.0,216700.0,95600.0,771100.0,...,80100.0,136300.0,78300.0,136600.0,62600.0,94300.0,92500.0,57300.0,192400.0,176300.0
1996-06,336500.0,236700.0,212200.0,503100.0,77300.0,95400.0,152600.0,216900.0,95800.0,776500.0,...,79400.0,136600.0,78200.0,136800.0,62700.0,94000.0,92400.0,57500.0,193700.0,176100.0
1996-07,337600.0,235400.0,210700.0,504600.0,77300.0,95700.0,152400.0,217000.0,96100.0,781900.0,...,78600.0,136900.0,78200.0,136800.0,62700.0,93700.0,92200.0,57700.0,195000.0,176000.0
1996-08,338500.0,233300.0,208300.0,505500.0,77400.0,95900.0,152300.0,217100.0,96400.0,787300.0,...,77900.0,137100.0,78100.0,136700.0,62700.0,93400.0,92100.0,58000.0,196300.0,175900.0
1996-09,339500.0,230600.0,205500.0,505700.0,77500.0,96100.0,152000.0,217200.0,96700.0,793000.0,...,77200.0,137400.0,78000.0,136600.0,62600.0,93200.0,91900.0,58200.0,197700.0,175800.0
1996-10,340400.0,227300.0,202500.0,505300.0,77600.0,96200.0,151800.0,217500.0,96800.0,799100.0,...,76400.0,137700.0,78000.0,136400.0,62500.0,93000.0,91700.0,58400.0,199100.0,175800.0
1996-11,341300.0,223400.0,199800.0,504200.0,77700.0,96100.0,151600.0,217900.0,96800.0,805800.0,...,75600.0,137900.0,78000.0,136000.0,62400.0,92900.0,91300.0,58700.0,200700.0,176000.0
1996-12,342600.0,219600.0,198300.0,503600.0,77700.0,96000.0,151600.0,218600.0,96700.0,814400.0,...,74900.0,138300.0,78100.0,135800.0,62300.0,92700.0,90900.0,59100.0,202600.0,176200.0
1997-01,344400.0,215800.0,197300.0,503400.0,77800.0,95800.0,151700.0,219700.0,96600.0,824300.0,...,74300.0,138700.0,78300.0,135800.0,62300.0,92600.0,90500.0,59500.0,204900.0,176500.0


In [42]:
df_t.index = pd.to_datetime(df_t.index)

In [50]:
# df_t["1997"].plot(figsize=(12,8))
# plt.show()

In [58]:
print ('Results of Dickey-Fuller Test:')
df_t_test = adfuller(df_t[0])

# Extract and display test results in a user friendly manner
dfoutput = pd.Series(df_t_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in df_t_test[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)


Results of Dickey-Fuller Test:


KeyError: 0